# Conv2D - MaxPooling


In [1]:
%matplotlib inline

import os
import sys
import certifi
import unicodedata
import pandas as pd
import numpy as np
import re
import csv
import math
import requests
import time
import random
import scipy.stats as stats
import matplotlib.pyplot as plt
import gensim
import keras
import h5py
import pydot
import os
import sklearn
import operator

from keras.layers.merge import Add
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical, plot_model
from keras.layers import Conv2D, MaxPooling2D, Embedding, Merge, LSTM, Bidirectional, GlobalMaxPooling1D, Dot, Lambda, Dropout
from keras.layers import Average, dot,Permute, Lambda,Layer, add, Concatenate, Dense, LSTM, Input, concatenate, merge, Add, Reshape, Flatten
from keras.models import Model, load_model
from keras.models import Sequential
from numpy import newaxis
from keras.models import model_from_json
from keras import callbacks
from keras.callbacks import ModelCheckpoint, EarlyStopping,CSVLogger
from collections import defaultdict
from scipy.stats.stats import pearsonr
from keras import backend as K
from sklearn.model_selection import StratifiedKFold
from keras import metrics

K.os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
K.os.environ["CUDA_VISIBLE_DEVICES"]="0"

Using TensorFlow backend.
/home/inversi/.conda/envs/TextProject/lib/python3.5/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# LOAD WORD2VEC MODEL

In [2]:
#LOAD GENSIM MODEL WITH WORD2VEC 
model_W2C = gensim.models.Word2Vec.load('model/vector_50/mymodelFull')

# load the whole embedding into memory
embeddings_index = dict()
f = open('wiki-w2v/it-vectors.50.5.50.w2v', encoding = "utf-8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))


Loaded 214690 word vectors.


In [3]:
# create a weight matrix for words in training docs
vocab_size = len(model_W2C.wv.vocab)
embedding_matrix = np.zeros((vocab_size, 50))
for i in range(len(model_W2C.wv.vocab)):
    embedding_vector = embeddings_index.get(model_W2C.wv.index2word[i])
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

# DATA TRAINING MANIPULATION

In [4]:
#LOAD TOKENIZER FILES
def loadSentences(filename):
    array = []
    with open(filename, encoding = "utf-8") as f:
        ar = []
        for line in f:
            if (str(line) != "[\n") & (str(line) != "]\n"):
                ar.append(re.sub('\n$', '', line))
                #print (line)
            if (str(line) == "]\n"):
                array.append(ar)
                ar = []
            
    f.close()
    return array

#ANSWER TRAINING
original_A_train = loadSentences("tokenizer/tokenizerAnswer.txt")
A_train = loadSentences("tokenizer/tokenizerAnswer.txt")

#QUESTION  TRAINING
original_Q_train = loadSentences("tokenizer/tokenizerQuestion.txt")
Q_train = loadSentences("tokenizer/tokenizerQuestion.txt")

ANSWER

In [5]:
print(A_train[3])

['Il', 'deposito', 'cauzionale', 'e', 'una', 'somma', 'di', 'denaro', 'che', "l'", 'utente', 'versa', 'al', 'gestore', 'a', 'titolo', 'di', 'garanzia', 'e', 'che', 'deve', 'essere', 'restituita', 'dopo', 'la', 'cessazione', 'del', 'contratto', 'nel', 'rispetto', 'delle', 'condizioni', 'contrattuali', 'in', 'vigore']


QUESTION

In [6]:
print(Q_train[3])

['Cosa', 'è', 'il', 'deposito', 'cauzionale']


In [7]:
#DATASET MANIPULATION FOR TRAINING
random_idx = [random.randrange(0, len(original_Q_train)) for i in range(0, len(original_Q_train))]

new = []
for i in random_idx:#len(A_train)):
    new.append(original_Q_train[i])
Q_train.extend(new)

new = []
for i in random_idx:#len(A_train)):
    new_random = random.randrange(0, len(random_idx))
    while (new_random == i):
        new_random = random.randrange(0, len(random_idx))
    new.append(original_A_train[new_random])
A_train.extend(new)

print(len(A_train))
print(len(Q_train))

812
812


In [8]:
#Y_train LABEL CLASS FOR EVALUATION TRAINING
Y_train = []
for i in range(0, (int(len(A_train)/2))):
    a=[]
    a.append(1)
    Y_train.append(a)
for i in range(int(len(A_train)/2+1), len(A_train) + 1):
    a=[]
    a.append(0)
    Y_train.append(a)
Y_train = np.array(Y_train)
print(len(Q_train),len(A_train),len(Y_train))

812 812 812


In [9]:
#SUBSTITUTE WORD WITH INDEX
def replace_matched_items(array, dictionary):
       for lst in array:
              for ind, item in enumerate(lst):
                          lst[ind] = dictionary.get(item, item)       

In [10]:
#CLEAN ZEROS
def cleaningZero(array):
       for elem in array:
            while 0 in elem: 
                elem.remove(0)

In [11]:
#vocab - uso lo stesso vocab
vocab = dict([(k, v.index) for k, v in model_W2C.wv.vocab.items()])

#REPLACE A_TRAIN AND Q_TRAIN WITH VOCAB INDEX
replace_matched_items(Q_train,vocab)
cleaningZero(Q_train)
replace_matched_items(A_train,vocab)
cleaningZero(A_train)

In [12]:
#PAD DOCUMENTS FOR QUESTION AND ANSWER
max_len = max(len(max(A_train,key=len)), len(max(Q_train,key=len)))
Q_train_padded = keras.preprocessing.sequence.pad_sequences(Q_train, padding='post', maxlen = max_len)
A_train_padded = keras.preprocessing.sequence.pad_sequences(A_train, padding='post', maxlen = max_len)
#print(Q_train_padded.shape, A_train_padded.shape,Y_train.shape)

# KERAS MODEL (INPUTS: Q - A)


In [13]:
#FUNCTON FOR MATRIX MULTIPLICATION
class OuterProduct(Layer):
    def __init__(self,name="OuterProduct",**kwargs):
        super().__init__()

    def build(self,input_shape):
        self.length = input_shape[0][1]
        super().build(input_shape)

    def call(self,x):
        a, b = x
#        print(a.shape,b.shape,a[:, newaxis, :].shape,b[:, :, newaxis].shape)
        outerProduct = a[:, newaxis, :] * b[:, :, newaxis]
 #       print(outerProduct.shape)
        outerProduct = K.sum(outerProduct,axis=-1)
  #      print(outerProduct.shape)

        return outerProduct

    def compute_output_shape(self, input_shape):
        return (input_shape[0][0], input_shape[0][1], input_shape[0][1])

In [18]:
#Save the model after every epoch. https://keras.io/callbacks/#modelcheckpoint
checkpoint = ModelCheckpoint("keras_models/Q_A_Conv2D_MaxPool.h5", monitor='val_acc', verbose=1, save_best_only=True, mode='max')
#Stop training when a monitored quantity has stopped improving. https://keras.io/callbacks/#earlystopping
#stop = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=1, mode='auto')
csv_logger = CSVLogger('history/CONV_Q-A/log.csv', append=True, separator=';')

callbacks_list = [checkpoint,csv_logger]

In [17]:
#LAYER EMBEDDING-BiLSTM FOR ANSWER
A_input = Input(batch_shape=(None,A_train_padded.shape[1],), name='A_input')
A = Embedding(len(vocab), output_dim=50, weights=[embedding_matrix],name="Embedding_A",trainable=True)(A_input)
A_LSTM = Bidirectional(LSTM(50, dropout=0.5, recurrent_dropout=0.5,return_sequences=True),name="LSTM_A")(A)


#LAYER EMBEDDING-BiLSTM FOR QUESTIONS
Q_input = Input(batch_shape=(None,Q_train_padded.shape[1],),  name='Q_input')
Q = Embedding(len(vocab),output_dim=50,name="Embedding_Q",trainable=True)(Q_input)
Q_LSTM = Bidirectional(LSTM(50, dropout=0.5, recurrent_dropout=0.5,return_sequences=True),name="LSTM_QA")(Q)

#PRODOTTO DELLE MATRICI EMB
M1 = OuterProduct()([A, Q])
M1 = Reshape((-1,A_train_padded.shape[1],Q_train_padded.shape[1]),name="M1_Embedding")(M1)
M2 = OuterProduct()([A_LSTM, Q_LSTM])
M2 = Reshape((-1,A_train_padded.shape[1],Q_train_padded.shape[1]),name="M2_LSTM")(M2)
M12 = Concatenate(axis=1,name="M1xM2")([M1, M2])


#CONVUTIONAL
Convo = Conv2D(8, 3, activation='relu', data_format='channels_first',name="Convolutional")(M12)
Convo = Dropout(.5)(Convo)

#MAXPOOLING
Pool = MaxPooling2D(pool_size=(2, 2), data_format='channels_first',name="Max_Pooling")(Convo)
Pool = Dropout(.5)(Pool)
Pool = Flatten()(Pool)

#MLP
dense = Dense(200, activation='relu')(Reshape((-1,))(Pool))
dense = Dropout(.5)(dense)
dense= Dense(100, activation='relu')(dense)
dense = Dropout(.5)(dense)

output = Dense(1,activation='sigmoid')(dense)

model = Model(inputs=[Q_input,A_input], outputs=output)
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy',metrics.mean_squared_error])

#model.summary(line_length=200)
print("Model Ready...")

Model Ready...


# PLOT PNG KERAS MODEL

In [ ]:
#PRINT MODEL LAYERS ON FILE.PNG
plot_model(model, to_file='image/Q_A_Conv2D_MaxPool.png', show_shapes=True, show_layer_names=True)

# TRAIN MODEL Q - A
with Stratified K-Fold with 10 splits

without strtified K-fold

In [20]:
history = model.fit([Q_train_padded,A_train_padded],Y_train, batch_size=25, epochs=80, validation_split=0.1,callbacks = callbacks_list)

Train on 730 samples, validate on 82 samples
Epoch 1/80
730/730 [==============================] - 132s - loss: 0.8300 - acc: 0.6712 - mean_squared_error: 0.2342 - val_loss: 0.7377 - val_acc: 0.5488 - val_mean_squared_error: 0.2709
Epoch 2/80
730/730 [==============================] - 131s - loss: 0.5822 - acc: 0.7301 - mean_squared_error: 0.1954 - val_loss: 0.9885 - val_acc: 0.4146 - val_mean_squared_error: 0.3667
Epoch 3/80
730/730 [==============================] - 131s - loss: 0.5091 - acc: 0.7644 - mean_squared_error: 0.1691 - val_loss: 1.3296 - val_acc: 0.3780 - val_mean_squared_error: 0.4463
Epoch 4/80
730/730 [==============================] - 131s - loss: 0.4594 - acc: 0.7932 - mean_squared_error: 0.1525 - val_loss: 1.4681 - val_acc: 0.4756 - val_mean_squared_error: 0.4481
Epoch 5/80
730/730 [==============================] - 131s - loss: 0.3938 - acc: 0.8411 - mean_squared_error: 0.1247 - val_loss: 1.8589 - val_acc: 0.4634 - val_mean_squared_error: 0.4640
Epoch 6/80
730/730 [

KeyboardInterrupt: 

In [ ]:
# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
plt.savefig(fname)

# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# summarize history for accuracy
plt.plot(history.history['mean_squared_error'])
plt.plot(history.history['val_mean_squared_error'])
plt.title('model mean_squared_error')
plt.ylabel('mean_squared_errory')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()